In [1]:
# Import necessary libraries
import ee
import geemap

# Authenticate to Earth Engine
# ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AfJohXk1j8Id94n7wMuLNyvClcw4zzeFShgnzMt6vcgdyHvxYlQ7EzqRrfg



Successfully saved authorization token.


In [21]:


# Define the FAO/GAUL administrative units dataset
dataset = ee.FeatureCollection('FAO/GAUL/2015/level2')

# Define a region of interest (ROI) for India
india_roi = ee.Geometry.Polygon(
    [[[
      68.11137880340998, 7.96553484576015],
      [88.3927306205192, 7.96553484576015],
      [88.3927306205192, 35.67459030295267],
      [68.11137880340998, 35.67459030295267],
      [68.11137880340998, 7.96553484576015]
    ]]
)


Map

Map(bottom=27621.0, center=[28.646455931994893, 77.10900424302301], controls=(WidgetControl(options=['position…

In [14]:
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

# Define the FAO/GAUL administrative units dataset
dataset = ee.FeatureCollection('FAO/GAUL/2015/level2')

# Define a region of interest (ROI) for India
india_roi = ee.Geometry.Polygon(
    [[[
      68.11137880340998, 7.96553484576015],
      [88.3927306205192, 7.96553484576015],
      [88.3927306205192, 35.67459030295267],
      [68.11137880340998, 35.67459030295267],
      [68.11137880340998, 7.96553484576015]
    ]]
)

# Filter the administrative units dataset to select India
india_dataset = dataset.filterBounds(india_roi)

# Define a region of interest (ROI) for New Delhi
delhi_roi = ee.Geometry.Point(77.216721, 28.613939).buffer(0.5)

# Filter the India dataset to select New Delhi
delhi_feature = india_dataset.filterBounds(delhi_roi)

# Print the selected administrative unit (New Delhi)
print("Selected Administrative Unit (New Delhi):")
selected_features = delhi_feature.getInfo()
for feature in selected_features['features']:
    print("Admin Unit ID:", feature['properties']['ADM2_CODE'])
    print("Admin Unit Name:", feature['properties']['ADM2_NAME'])

# Create a Map
Map = geemap.Map()

# Set the Map center
Map.setCenter(12.876, 42.682, 5)

# Add the administrative units dataset to the map
Map.centerObject(india_dataset, 5)
Map.addLayer(india_dataset, {'color': '00909F'}, 'Second Level Administrative Units')

# Display New Delhi
Map.centerObject(delhi_feature, 8)
Map.addLayer(delhi_feature, {'color': 'FF0000'}, 'New Delhi')

# Display the Map
Map


Selected Administrative Unit (New Delhi):
Admin Unit ID: 17633
Admin Unit Name: Delhi


Map(center=[28.646455931994893, 77.10900424302301], controls=(WidgetControl(options=['position', 'transparent_…

In [27]:
# Filter the administrative units dataset to select administrative 2 regions surrounding Delhi
delhi_surrounding_admin2 = india_dataset.filterBounds(delhi_roi).filter(ee.Filter.eq('ADM2_CODE', '2'))

# Clip the study area to the administrative units around Delhi
clipped_study_area = india_dataset.filterBounds(delhi_surrounding_admin2)

# Export the clipped study area as a vector shapefile
export_params = {
    'collection': clipped_study_area,
    'description': 'Study_Area',
    'fileFormat': 'SHP'  # Export as a shapefile
}

# Define the export task
export_task = ee.batch.Export.table.toDrive(**export_params)

# Start the export task
export_task.start()

# Wait for the export task to complete
import time
while export_task.status()['state'] == 'RUNNING':
    time.sleep(5)

# Check if the export task completed successfully
if export_task.status()['state'] == 'COMPLETED':
    # Get the URL of the exported file
    export_url = export_task.status()['destinationUrl']
    print("File saved at:", export_url)
else:
    print("Export task failed:", export_task.status())


Export task failed: {'state': 'READY', 'description': 'Study_Area', 'creation_timestamp_ms': 1696419400819, 'update_timestamp_ms': 1696419400819, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_FEATURES', 'id': 'IW6SYI2S7A7RBQVDASWTI7ZH', 'name': 'projects/earthengine-legacy/operations/IW6SYI2S7A7RBQVDASWTI7ZH'}


In [28]:
# Start the export task
export_task.start()

# Wait for the export task to complete
import time
while export_task.status()['state'] == 'RUNNING':
    time.sleep(30)  # Increased delay to 30 seconds
    print("Export task is still running...")

# Check if the export task completed successfully
if export_task.status()['state'] == 'COMPLETED':
    # Get the URL of the exported file
    export_url = export_task.status()['destinationUrl']
    print("File saved at:", export_url)
else:
    print("Export task failed:", export_task.status())

EEException: A different Operation was already started with the given request_id.